In [1]:
# import neccessary dependencies
from citipy import citipy
import gmaps
from config import g_key,weather_api_key
import numpy as np
import pandas as pd
import requests

In [2]:
#generate 2000 random lats and longs
lats = np.random.uniform(-90,90,size=2000)
long = np.random.uniform(-180,180,size=2000)
coords = list(zip(lats,long))

In [3]:
# get the nearest cities for each coordinate pair 
cities = []
for coordinate in coords:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
print(len(cities))
len(cities)

750


750

In [4]:
# create a DataFrame with the city names
cities_df = pd.DataFrame(cities)
cities_df.columns = ["City"]

# define a function that will be applied to the df which calls the api and fills out the fields
def get_weather(row):
    city = row["City"]
    base_url = "https://api.openweathermap.org/data/2.5/weather?"
    params = {
        'units' : "imperial",
        'q' : city,
        'appid' : weather_api_key
    }
    try :
        print(f"Processing | {city}")
        r = requests.get(base_url,params).json()
        row["Lat"] = r["coord"]["lat"]
        row["Lng"] = r["coord"]["lon"]
        row["Country"] = r["sys"]["country"]
        row["Max Temp"] = r["main"]["temp_max"]
        row["Humidity"] = r["main"]["humidity"]
        row["Cloudiness"] = r["clouds"]["all"]
        row["Wind Speed"] = r["wind"]["speed"]
        row["Current Description"] = r["weather"][0]["description"]
    except KeyError as keyEx:
        print("City not found... skipping")
    return row
# apply the function to the dataFrame
cities_df = cities_df.apply(get_weather, axis=1)
#drop any rows that were not found
cities_df = cities_df.dropna()
cities_df.head()

Processing | khatanga
Processing | khatanga
Processing | ilulissat
Processing | punta arenas
Processing | attawapiskat
City not found... skipping
Processing | puerto ayora
Processing | peniche
Processing | cape town
Processing | ushuaia
Processing | victoria
Processing | college
Processing | kavieng
Processing | ponta do sol
Processing | lasem
Processing | qaanaaq
Processing | barrow
Processing | jamestown
Processing | atuona
Processing | bontang
Processing | bredasdorp
Processing | portland
Processing | lorengau
Processing | hermanus
Processing | tocopilla
Processing | samusu
City not found... skipping
Processing | kruisfontein
Processing | klaksvik
Processing | saldanha
Processing | bergen
Processing | cidreira
Processing | arraial do cabo
Processing | guerrero negro
Processing | kapaa
Processing | takoradi
Processing | bengkulu
Processing | taolanaro
City not found... skipping
Processing | ayagoz
Processing | sao joao da barra
Processing | illoqqortoormiut
City not found... skipping

Processing | san juan de la maguana
Processing | puerto carreno
Processing | bejar
Processing | coulihaut
City not found... skipping
Processing | mastic beach
Processing | newtownards
Processing | xuddur
Processing | bilma
Processing | umm kaddadah
Processing | jalu
Processing | pisco
Processing | novikovo
Processing | roura
Processing | tongliao
Processing | tommot
Processing | cotonou
Processing | tondano
Processing | whitianga
Processing | roebourne
Processing | izumo
Processing | igarka
Processing | guiratinga
Processing | half moon bay
Processing | gaptsakh
Processing | thompson
Processing | ahipara
Processing | lengshuitan
Processing | nanortalik
Processing | cantaura
Processing | colares
Processing | scarborough
Processing | mount gambier
Processing | sicamous
Processing | asfi
Processing | lufilufi
Processing | palmerston
Processing | esperance
Processing | salinopolis
Processing | pitkyaranta
Processing | berezovyy
Processing | skibbereen
Processing | morgan city
Processing | 

Processing | kedrovyy
Processing | thinadhoo
Processing | nyurba
Processing | high level
Processing | panguipulli
Processing | pangnirtung
Processing | upata
Processing | san cristobal
Processing | khonuu
City not found... skipping
Processing | romilly-sur-seine
Processing | murgab
Processing | chute-aux-outardes
Processing | altay
Processing | sorland
Processing | yerofey pavlovich
Processing | nouadhibou
Processing | kembe
Processing | ozgon
City not found... skipping
Processing | bandarbeyla
Processing | mount isa
Processing | shulin
Processing | zabid
Processing | waw
Processing | ronne
Processing | ucluelet
Processing | rennes
Processing | tiarei
Processing | kytlym
City not found... skipping
Processing | aksarka
Processing | mutsamudu
City not found... skipping
Processing | rostovka
Processing | anadyr
Processing | port hedland
Processing | palmer
Processing | chumikan
Processing | zlatoustovsk
City not found... skipping
Processing | itapirapua
Processing | seoul
Processing | sur

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
0,khatanga,100.0,RU,overcast clouds,99.0,71.9667,102.5000,-10.61,4.32
1,ilulissat,90.0,GL,snow,86.0,69.2167,-51.1000,33.80,10.36
2,punta arenas,0.0,CL,clear sky,71.0,-53.1500,-70.9167,46.40,14.97
4,puerto ayora,72.0,EC,broken clouds,91.0,-0.7393,-90.3518,79.00,8.01
5,peniche,100.0,PT,overcast clouds,87.0,39.3558,-9.3811,59.00,1.01


In [7]:
cities_df = cities_df[["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]]
cities_df.to_csv("WeatherPy_Database.csv",index_label="City_ID")